## Changing the XML files to simple TXT files

In [1]:
#importing all the necessary packages
import os
import xml.etree.ElementTree as ET
import csv
import pandas as pd

In [12]:
# Directory containing the XML files
xml_directory = '/kaggle/input/faz2018/'

# Initialize the list to store the extracted titles
titles = []

# Loop through all XML files in the directory
for filename in os.listdir(xml_directory):
    if filename.endswith(".xml"):
        file_path = os.path.join(xml_directory, filename)
        
        try:
            # Parse the XML file
            tree = ET.parse(file_path)
            root = tree.getroot()
            
            # Extract the "titel" attribute from the root element and add it to the titles list
            title_attribute = root.get("titel")
            
            # Extract the "untertitle" attribute from the root element
            subtitle_attribute = root.get("untertitel")
            
            if title_attribute:       
                full_title = title_attribute
                if subtitle_attribute:
                    full_title += ' ' + subtitle_attribute
                # Add the combined title to the titles list
                titles.append(full_title)
            else:
                print(f"Warning: 'titel' attribute not found in {filename}")
        
        except ET.ParseError:
            # Handle cases where XML parsing fails
            print(f"Error parsing {filename}")

# Print or use the list of extracted titles
for title in titles[:10]:
    print(title)

Abschiebung von Yousif A. war möglich Kretschmer: Bund ist zuständig / Strobl und Klingbeil fordern Beobachtung der AfD
Seehofer mahnt Merkel CSU-Vorsitzender fordert Unterstützung für Ankerzentren
17 Migranten ertrunken Bis November 100000 Migranten über das Mittelmeer
Nadelöhr Brenner Südtirol kritisiert das neue österreichische "Grenzmanagement" / Von Tobias Lechner
Amerika verringert Aufnahme
Unterbringung in der grünen Pampa
Rechte und Pflichten
Sanktionen gegen Schleuser
Wohnungen für Alte und Flüchtlinge
Kritik an Flüchtlingsunterkunft Stadt stellt Pläne bei Veranstaltung in Oberrad vor


## Extracting the nouns

In [3]:
#downloading the large German model
!python -m spacy download de_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.8/567.8 MB 851.8 kB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [4]:
import spacy

In [5]:
#loading the German model in spaCy
nlp = spacy.load('de_core_news_lg')

In [13]:
#finding the words that have the POS tags for nouns
# Initialize the list to store the extracted nouns
nouns = []

# Process each title and extract nouns
for title in titles:
    doc = nlp(title)
    for token in doc:
        if token.pos_ == "NOUN":
            nouns.append(token.text)
         
        #the tagger finds 670 nouns, while the pos finds 667 nouns 
        #-> pos is more conservative so that's the one i'm using
        #if token.tag_ in ['NN']:
            #nouns.append(token.text)

# Print or use the list of extracted nouns
print(len(nouns))
print(nouns[:10])

2067
['Abschiebung', 'Bund', 'Beobachtung', 'CSU-Vorsitzender', 'Unterstützung', 'Ankerzentren', 'Migranten', 'November', 'Migranten', 'Grenzmanagement']


## Detecting the compounds 

In [14]:
# Create a new list to store the filtered nouns
long_nouns = []

# Loop through the list of nouns
for item in nouns:
    # Check if the length of the noun is at least 7 characters
    if len(item) >= 7:
        # If it meets the condition, add it to the filtered_nouns list
        long_nouns.append(item)

# The filtered_nouns list will now only contain nouns with a length of 7 or more characters
print(len(long_nouns))
print(long_nouns[:10])

1548
['Abschiebung', 'Beobachtung', 'CSU-Vorsitzender', 'Unterstützung', 'Ankerzentren', 'Migranten', 'November', 'Migranten', 'Grenzmanagement', 'Aufnahme']


## Manual removal of non-compounds

In [16]:
#removing simplex nouns from the list of long_nouns
items_to_remove = ['Flüchtling', 'Flüchtlinge', 'Flüchtlingen', 'Polizei','Potenzial', 'Richter', 'Schleuser', 'Angriffe', 'Zuwanderung','Zuwanderer', 'Humanität', 'Zeichen', 'Paragrafen', 'Ungewisse','Betreiber', 'Übergriffe', 'Kollaps', 'Desaster', 'Proteste', 'Eindringlinge', 'Selbstständigkeit', 'Gedränge', 'Schande', 'Willkommen', 'Bosporus', 'Versicherung', 'Ländern', 'Gerüchte', 'Migranten', 'Freiheit', 'Freiheiten', 'Hungern', 'Ansturm', 
                   'Wirklichkeit', 'Hundert', 'Hunderte', 'Freiheit', 'Grenzen', 'Wunsch', 'Wünsche', 'Unsolidarischen', 'Rettung', 'Koalition', 'Debatte', 'Skandieren', 'Abschiebungen', 'Abschiebung', 'Ablehnung', 'Gefängnis', 'Verbündete', 'Weggesperrt', 'Minister', 'Wohunung', 'Wohnungen', 'Politik', 'Politiker', 'Politikerin', 'Nachdenken', 'Mission', 'Problem', 'Probleme', 'Unterstützung', 'Zentrale', 'Rassismus', 'Million', 'Millionen', 'Milliarde', 'Milliarden', 'Zwischengeparkte', 'Menschen', 'Tausend','Tausende', 'Provokation', 'Regierung', 'Bedauern', 'Stimungsmache', 'Verantwortung', 'Forderungen', 'Hoffnung', 'Schweizer', 'Aufstand', 'Zuflucht', 'Mitarbeiter', 'MANNHEIMER', 'Unterkunft', 'Hoffnung',
                   'Anzeige', 'Homosexuelle', 'Homosexueller', 'Sternen', 'Afrikaner', 
                   'Katastrophe', 'Unrechts', 'Anträge', 'Regiment', 'Ausländer', 
                   'Verdacht', 'Überfahrt', 'Einsatz', 'Spiegel', 'Abgeschoben', 
                   'Behörde', 'Angriff', 'Entlastung', 'Kirche', 'Kirchen', 
                   'Rasiermesserscharf', 'Gastlichkeit', 'Monstrum', 'Reaktionen', 
                   'Soldaten', 'Aufklärung', 'Geschäft', 'Geschacher', 'Gedenken', 
                   'Solidarität', 'Vorwürfe', 'September', 'Zeitung', 'Aufnahme', 
                   'Diskriminierte', 'Unbarmherzigen', 'Zustände', 'Unschuld', 
                   'Verfahren', 'Gefragte', 'Geschäfte', 'Warnung', 'Herausforderungen',
                   'Toiletten', 'Schlepper', 'Strände', 'Leichen', 'Einwanderung', 
                   'Anschlag','Zweifel', 'Routine', 'Fahndung', 'Unterkünfte', 'Kabinett',
                   'Vorurteile','Gestatten', 'Nachbarn', 'Bekenntnis', 'Sicherheit', 
                   'Kapelle', 'Zuhörer', 'AKTUELLEM', 'SCHWEIZ', 'Verständnis',
                   'Abgeschoben', 'Zustrom', 'Komunen', 'Berliner', 'Havarie',
                   'Stimmungsmache', 'Mobilmachung', 'Oranienplatz', 'Kommunen',
                   'Kommune', 'Rechtsradikaler', 'Selbständigkeit', 'Kompromiss',
                   'Erleichterung', 'Karte', 'Karten', 'Demonstranten', 'Abschrekende',
                   'Wirkung', 'Muslime', 'Stellen', 'Stelle', 'Chancen', 'Barrikaden', 
                   'Richtung', 'Brenner', 'Feilscherei', 'Argumente', 'Gewerkschaft',
                   'Staat', 'Staaten', 'Wirrungen', 'Wachstum', 'Telefon', 'Freunde', 
                   'Partner', 'Zukunft', 'Verzweiflung', 'Brandstiftung', 'Technik', 
                   'Unbekannten', 'Salafist', 'Salafisten', 'Klassen', 'Klasse', 
                   'Seehofer', 'Massaker', 'Immigration', 'Zündeln', 'Gewinne',
                   'Vorwurf', 'Polizisten', 'Aufgabe', 'Reserven', 'Gemeinden',
                   'Familie', 'Sexualität', 'Sabotage', 'Minderheit', 'Lamento',
                   'Nirgendwo', 'Standards', 'Schritt', 'Kosovaren', 'Empörung',
                   'Zukunft', 'Abweisung', 'Abweisungen', 'Minderjährige', 'Versuch',
                   'Praktikum', 'Praktika', 'Philanthrop', 'Standard', 'Einreise',
                   'NACHRICHTEN', 'Provokateur', 'Schleusern', 'Dilemma', 'Dschungel',
                   'Anspruch', 'Auflage', 'Auflagen', 'Messerstecherei', 'Messerstecherein',
                   'Paradox', 'Gewöhnung', 'Anführer', 'Schwerte', 'Tonnenweise',
                   'Verdächtige', 'Rechnung', 'Drohungen', 'Verdächtiger', 'Anschläge',
                   'Schläge', 'Zentrum', 'Rechter', 'Treffen', 'Arbeiteslose', 'Schwaetzer',
                   'Arbeitesloser', 'Betreuung', 'Politbarometer', 'Tragödie', 'Demokratie',
                   'Militär', 'Militärs', 'Integration', 'Gesundheit', 'Wirtschaft',
                   'Visionen', 'Basteln', 'Unbekanntem', 'Ignoranten', 'Gutscheine', 
                   'Gutschein', 'Bedrängnis', 'Werbung', 'Ausbildung', 'Einsätze',
                   'Selfies', 'Abfertigung', 'Wahnsinn', 'Gesellschaft', 'Zehntausenden', 
                   'Flammen', 'Befreiung', 'Verlorenen', 'Konjunktur', 'Pornografie',
                   'Republik', 'Einhundertsechzigtausend', 'Sprache', 'Schule',
                   'Schulen', 'Umfrage', 'Ausweisung', 'ALLGEMEINE', 'Kanzlerin',
                   'Gericht', 'Prinzip', 'Bewährung', 'Afghanen', 'Arbeitslose',
                   'Moschee', 'Durchschaufen', 'Verlogen', 'Populisten', 'Festnahme',
                   'Festnahmen', 'Zuständigkeiten', 'Fataler', 'Zugeständnisse', 
                   'Katholiken', 'Ffffffft', 'Abmarsch', 'Einwanderer', 'Tritten',
                   'Arabisch', 'Balance', 'Sächsin', 'Anwerben', 'Verbindung', 
                   'Gesellschaft', 'Wurstigkeit', 'Entscheidung', 'Schwächen', 'Migration',
                   'Schüsse', 'Freunden', 'Schelte', 'Ungezählte', 'Gezählte',
                   'Gesellschaft', 'Distanz', 'Schlägerei', 'Verletzte', 'Versprechen',
                   'Gerechtigkeit', 'Schatten', 'Belohnung', 'Vorfall', 'Dörfler', 'Deutsch',
                   'Klausur', 'Basteleien', 'Mahnung', 'Fischen', 'Verzweifelten', 'Ausweis',
                   'Auswanderung', 'Anzeiger', 'Distanz', 'Stimmung', 'Anwerber',
                   'Zusammenarbeit', 'Kontrolle', 'Kritikern', 'Rätselhafter', 
                   'Verheißung', 'Magazin', 'Verluste', 'Rechtsradikale', 'Versöhnung',
                   'Behörden', 'Komplett', 'Betrüger', 'Verbundenheit', 'Ermittlungen',
                   'klopfet', 'Einwanderern', 'Verhinderte', 'Konsens', 'Schlange', 
                   'Quadratur', 'Gefühle', 'Opposition', 'Schulden', 'Franken', 'Erlenweg',
                   'Jesiden', 'Nachrichten', 'Prüfung', 'Ausweisen', 'Verwaltung', 'Minimum',
                   'Kontrollen', 'Überzeugung', 'Vermietern', 'Einmauern', 'Hospodarske', 
                   'Konsulaten', 'Dankeschön', 'Terrorist', 'Pädagogik', 'Geretteten', 
                   'Österreicher', 'Einigung', 'Applaus', 'Demokratien', 'Annäherung',
                   'Courage', 'Wahrheit', 'Prügelei', 'Sekunde', 'Bedrohung', 
                   'Scherben', 'Verlierern', 'Gewinner', 'Präsidenten', 'Protest',
                   'Verhalten', 'Bereitschaft', 'Konzessionen', 'Erbarmen', 'Zutritt',
                   'Heuchler', 'Verteilung', 'Sitzung', 'Gescheucht', 'Übergang',
                   'Empfang', 'Vorstoß', 'Kameras', 'Minuten', 'Entscheider'
                   'Trennung', 'Herkunft', 'Geständnis', 'Korridor', 'Flüchtende',
                   'Deutsche', 'Perspektive', 'Menschlichkeit', 'Abschied', 'Durchatmen', 
                   'Hoffnungen', 'Aktionismus', 'Stunden', 'Mitteln', 'Elenden',
                   'Pflicht', 'Rechtsextremistischer', 'Verbrechen', 'Bremser',
                   'Uneinigkeit', 'Kratzer', 'Möglichkeiten', 'Kontinente', 'Strecke',
                   'Gesichter', 'Attacken', 'Spektakel', 'Koordinator', 'Beifall', 
                   'Abschaffen', 'Beschämende', 'Pauschale', 'Europäer', 'Vergleiche', 
                   'Respekt', 'Berlingske', 'Mutigen', 'Renationalisierung',
                   'Skeptikers', 'Konfrontation', 'Ausnahme', 'Beispiel',
                   'POLITBAROMETER', 'Affront', 'Petitesse', 'Papiere', 'Störung',
                   'NOVEMBER', 'Deutsche', 'Investition', 'Krawallen', 'Ertrunken',
                   'Kapital', 'Zerwürfnis', 'Leidenschaft', 'Debatten', 'Bürokratie',
                   'Entsorgung', 'Signale', 'Prügeleien', 'Blockaden', 'Europäer',
                   'Ermittlung', 'Korruption', 'Anstieg', 'Egoismus', 'Zehntausende',
                   'Integrieren', 'Attacke', 'Buddhisten', 'Schießerei', 'Stückchen',
                   'Knöcheltief', 'Schlamm', 'Attacken', 'Stiefel', 'Bischöfe', 'Vorurteil',
                   'Skepsis', 'Rechten', 'Annonce', 'Potenziale', 'Schwarze', 'Bereich',
                   'Schweigen', 'Herzlichkeiten', 'Einheit', 'Schweigen', 'Mindeste', 
                   'Verlogenheit', 'Maschen', 'Schlüssel', 'Kontroverses', 'Zuwachs', 'Stimmen',
                   'Schlägereien', 'Lösungen', 'Priorität', 'Präsident', 'Aufschwung', 'Schnitt',
                   'Kloster', 'Freiwillige', 'Neonazis', 'Exzesse', 'Euphorie', 
                   'Abschreckung', 'Vorhang', 'Kontingente', 'Gewissheiten', 'Vofeld',
                   'Randale', 'Ausbeuter', 'Papieren', 'Gutachten', 'Märchen', 
                   'Ausreisen', 'Granate', 'Aussicht', 'Verrückter', 'Anhörung', 
                   'Beschneidung', 'Vertrauen', 'Versagen', 'Freikaufen', 'Parcours', 
                   'Islamisten', 'Studentin', 'Zugriff', 'Türkisch', 'Anfänger', 
                   'Kanzler', 'Aufgewühlte', 'Rückkehr', 'AUSBILDER', 'Zänkereien', 
                   'Dörfern', 'Referendum', 'Situation', 'Ablenker', 'Entwarnung', 
                   'Bedingung', 'Bedingungen', 'Schlangen', 'Passagiere', 'Perspektiven', 
                   'Vergewaltigung', 'Gerücht', 'Gerüchte', 'Klischee', 'Verlierer', 
                   'Gezerre', 'Blockade', 'Personal', 'Bischof', 'Informanten', 
                   'Schwamm', 'Unbehagen', 'Dolmetscher', 'Dolmetschers', 'Lieblinge', 
                   'Aufgeben', 'Abschieben', 'Produkt', 'Dschihadist', 'Blockade', 
                   'FEBRUAR', 'Angebot', 'Beitrag', 'Rechtsextremen', 'Gipfelchen', 
                   'Willigen', 'Realität', 'Widerstand', 'Beschlüsse', 'Fronten', 
                   'Kapazitäten', 'Öresund', 'Hinschauen', 'Abkühlung', 'Beschluss', 
                   'Referendum', 'Nachbarschaft', 'Missfallen', 'Anzeigen', 'Kündigung', 
                   'Schleusung', 'Versagens', 'Griechen', 'Auftrag', 'Misstrauen', 
                   'Nötigste', 'Vergessenen', 'Kriterien', 'Symptom', 'Wiederentdeckung', 
                   'Herrschaft', 'Prozent', 'Kreuzer', 'Kontext', 'Vernunft', 'anderen', 
                   'Isolation', 'Anbieter', 'Statistik', 'Analphabeten', 'Fragile', 
                   'Fortschritt', 'Schüler', 'Klarheit', 'Pflichten', 'Unwillkommenen', 
                   'Pragmatismus', 'Geiseln', 'Schuldige', 'Unschuldige', 'Beschwerde', 
                   'Parlament', 'Dolmetscher', 'Angehörige', 'Extremisten', 'Räumung', 
                   'Langeweile', 'Krawalle', 'Stabilisierung', 'Hoffnungslosigkeit', 
                   'Begehrlichkeiten', 'Bestrafte', 'Expedition', 'Nachsitzen', 
                   'Genossen', 'Therapie', 'Verständigung', 'Töpfchen', 'Samariter', 
                   'Attentat', 'Szenario', 'Prozess', 'Rätseln', 'Kommissar', 'Absturz', 
                   'Herausforderung', 'Attentäter', 'Kriminalität', 'Einigkeit', 
                   'Identitäre', 'Nörgeln', 'Entwarner', 'Monsieur', 'Ausreise', 
                   'Zuschauer', 'Häufigkeit', 'Erpressung', 'Unterkünften', 'Mechanik', 
                   'Kooperation', 'Prognose', 'Mehrheit', 'Deutschen', 'Marokkaner', 
                   'Nickeleien', 'Blamage', 'Container', 'Baracken', 'Grausamkeiten', 
                   'Ausnahmen', 'Ablenkung', 'Gesetze', 'Konflikte', 'Investitionen', 
                   'Schimpfen', 'Disziplin', 'Vorbehalte', 'Audienz', 'Räumung', 
                   'Panikmache', 'Schwule', 'Durchlass', 'Vecernjilist', 'Integrativ', 
                   'Provinz', 'Konflikt', 'Schwierigkeiten', 'Toleranz', 'Forschung', 
                   'Unternehmer', 'adevarul', 'Enttäuschung', 'Umfragen', 'Integrator', 
                   'Hindernissen', 'Zerwürfnisses', 'Kampagne', 'Ablenkung', 'Fetisch',
                   'Ohrfeige', 'Nerstheimer', 'Rotgefährdet', 'Diplomaten', 'Delikte', 
                   'Aussichten', 'Entscheider', 'Knüppel', 'Untergang', 'Abgelehnte', 
                   'Ertrunkenen', 'Boulevards', 'Ignoranz', 'Adevarul', 'Schlechten', 
                   'Unverschämtheit', 'Appelle', 'Hemmungslosen', 'Vereinbarung', 
                   'Initiative', 'Kretschmann', 'Hartherzigkeit', 'Zermürbten', 
                   'Austausch', 'Abschauen', 'Vorsprung', 'Reportagen', 'Nachhinein', 
                   'Geschmack', 'Beladene', 'Renovierung', 'Dschungels', 'Loyalität', 
                   'INDEPENDANT', 'Definitionen', 'Strafen', 'Duldung', 'Strapaze', 
                   'Verzicht', 'Unpäpstlicher', 'Pförtner', 'Arbeiten', 'Condrobs', 
                   'Viechtacher', 'Anliegen', 'Fraktion', 'Nerstheimer', 'Rücknahme', 
                   'Paranoia', 'Flüchtlings', 'Feindseligkeit', 'Tunnels', 'Schleppender', 
                   'Dissens', 'Gewässer', 'Gewässern', 'Geschlossenheit', 'Kommission', 
                   'Konferenz', 'Menetekel', 'Schlafen', 'Ergebnis', 'Psychiatrie', 
                   'Organisation', 'Palästinenser', 'Pessimismus', 'Parteien', 
                   'Einbrüche', 'Maschine', 'Dienstag', 'Besorgnis', 'Forderung', 
                   'Nachhall', 'Statistiken', 'Gelenkte', 'Marathons', 'Zivilisten', 
                   'Drohung', 'Betrügern', 'Vermisste', 'Schätzungen', 'Abschluss', 
                   'Experten', 'Umdenken', 'Vertriebene', 'Früchte', 'Gespräch', 
                   'Abwartens', 'Afghane', 'Anschlags', 'Hunderttausende', 'Berichte', 
                   'Misshandlungen', 'Anhängern', 'Begründung', 'Identität', 
                   'Ausschreitungen', 'Exklave', 'Betroffenen', 'Beobachter', 
                   'Ausland', 'Politologin', 'Brücken', 'Mitgliedschaft', 
                   'Angehörigen', 'Erfahrungen', 'Geflüchteten', 'Trichter', 
                   'Entscheidungen', 'Ermittler', 'Konzept', 'Bewohner', 
                   'Rückführung', 'Ankömmlinge', 'Fleisch', 'Angestaute', 
                   'Überlebende', 'NZZamSonntag', 'Nachricht', 'Schrauben', 
                   'Erteilung', 'Familien', 'Migrantinnen', 'Ausbildungen', 
                   'Unverständnis', 'Konsequenzen', 'Demokraten', 'Einstellungen', 
                   'Positionen', 'Parteien', 'Verrohung', 'Schwebe', 'Generation', 
                   'Registrierung', 'Forscher', 'Anmerkungen', 'Geflüchtete', 
                   'Individualität', 'Partizipien', 'Vereinfachung', 'Diskriminierung', 
                   'Brigade', 'Nationen', 'Zunahme', 'Vergleich', 'Polizist', 
                   'Rohingyas', 'Teilhabe', 'Scharfmacher', 'Premier', 
                   'Arbeitslosen', 'Schwächsten', 'Anarchie', 'Berater', 
                   'Unqualifizierte', 'Diskussion', 'Gerangel', 'Mitsprache', 
                   'Heikler', 'Fenster', 'Reinkommen', 'Hindukusch', 'Theorie', 
                   'Konservative', 'Programm', 'Unglück', 'Abgeordneten', 
                   'Miteinander', 'Geschichte', 'Betriebe', 'Friseurin', 'Slowaken', 
                   'Kennenlernen', 'Gefährder', 'Wechsel', 'Auslesen', 'Schreckens', 
                   'Arbeitslos', 'Säuberungen', 'Zugezogene', 'Abkommen', 
                   'Entführung', 'Diskussionen', 'Zuspruch', 'Unterbringung', 
                   'Entwicklung', 'Projekt', 'Ablehnungen', 'Monaten', 
                   'Verlängerung', 'Schwarz-Grün', 'Frühling', 'Bewaffnete', 
                   'Kämpfen', 'Zusammenleben', 'Ursachen', 'Regionen', 'Nachfolger', 
                   'Bescheidenheit', 'Umstände', 'Umverteilung', 'Störrische', 
                   'Freizügigkeit', 'Schwaben', 'Südsudanesen', 'Theorie', 'Krawall', 
                   'Papstes', 'Unterhändler', 'ZEITUNG', 'Petition', '15-Jährige', 
                   'Überblick', 'Vereinigung', 'Prozesse', 'Moratorium', 'Beamtin', 
                   'Bewerber', 'Betriebe', 'Tunesier', 'Palästinensern', 'Begründungen', 
                   'Vernetzung', 'Betreuten', 'Problemen', 'Ehrenamtliche', 'Verarbeitung', 
                   'Ausstellung', 'Ankommens', 'Archive', 'Skandal', 'Programm', 
                   'Betreuer', 'Zahlungen', 'Muslimen', 'Antisemitismus', 'Geflohener', 
                   'Revision', 'Plädoyer', 'Kriminelle', 'Einschränkung', 'Anstrengungen', 
                   'Heimkehr', 'Fortschritte', 'Arbeiter', 'Hinweise', 'Unglücks', 
                   'Ursache', 'Bestand', 'Ausweisungen', 'Rückkehrer', 'Schwächsten', 
                   'Paragraf', 'Atheistin', 'Einblick', 'Entkommen', 'Deserteur', 
                   'Ehrenamtliche', 'Streiks', 'Briefen', 'Resolutionen', 'Misshandlung', 
                   'Anklagen', 'Begrenzter', 'Christentum', 'Verdächtiger', 'Prostituierte', 
                   'Einblick', 'Chronik', 'Beschränkung', 'Aktivisten', 'Schulungen', 
                   'Wiederkehr', 'Details', 'Engagement', 'Pfarrer', 'Rückführungen', 
                   'Offizier', 'Anklage', 'Prinzipien', 'Verschieben', 'Bericht', 'Absprachen', 
                   'Energie', 'Personen', 'Adressen', 'Erfahrung', 'Unternehmen', 'Lehrlinge', 
                   'Dossier', 'Ankünfte', 'Willens', 'Vergewaltigungen', 'Schlüssen', 'Kriminologe', 
                   'Digitalisierung', 'Entspannung', 'Lehrling', 'Botschafter', 
                   'Erhöhung', 'Behandlung', 'Künstler', 'Quartal', 'Anerkennung',
                   'Überbelegung', 'Beteiligte', 'Selbstgerechtigkeit', 'Sendlinger-Tor-Platz', 
                   'Überlebenden', 'Vorschlag', 'Zugeständnissen', 'Mittelfranken', 
                   'Rechtsextremist', 'Anmeldung', 'Kliniken', 'Verfolgten', 'Hoffnungslosen', 
                   'Münchner', 'Kräften', 'Verfolgung', 'Minderheiten', 'Beschwerden', 
                   'Standorte', 'Beschuss', 'Wohnung', 'Aufträge', 'Kasernen', 'Verfügung', 
                   'Freundschaft', 'Verbesserungen', 'Verbesserung', 'Bibliothek', 'Maßnahme', 
                   'Maßnahmen', 'Antworten', 'Anstalten', 'Unbekannte', 'Krankheit', 
                   'Kapuzinerhölzl', 'Versäumnissen', 'Alternative', 'Aufsteiger', 
                   'Verfügung', 'Gefallen', 'Vertreter', 'Versorgung', 'Vertriebenen', 
                   'Schicksale', 'Wirrnisse', 'Gemeinschaft', 'Demonstration', 'Mitnehmen', 
                   'Drittel', 'Södertäljes', 'Assyrer', 'Verdächtigen', 'Zuständigkeit', 
                   'Zusammenstöße', 'Christen', 'Verbände', 'Kenntnis', 'Gartlberg', 'Schicksal', 
                   'Wissenschaftler', 'Sprachen', 'Kulturen', 'Ideologien', 'Anfeindungen', 
                   'Ausländern', 'Ukrainer', 'Ausbruch', 'Politikern', 'Eingreifen', 'Unglücken', 
                   'Geschichten', 'Anwohner', 'Agitation', 'Strategien', 'Ministerin', 'Aufwand', 
                   'Einrichtungen', 'Menschenrechtler', 'Botschaft', 'Botschaften', 
                   'Abschreckende', 'Initiativen', 'Hunderten', 'Afrikanern', 
                   'unrecht', 'Moderation', 'Optimismus', 'Schreiben', 'Begrenzung', 
                   'Schiffe', 'Erstaunlich', 'Ankunft', 'Attentate', 'Haltung', 
                   'Balkans', 'Kontakt', 'Anweisung', 'Übergriff', 'Gestrandeten', 
                   'Versicherer', 'Obdachlosen', 'Waggons', 'Überalterung', 'Stagnation', 
                   'Vereinigungen', 'Verschärfung', 'Vorhersage', 'Unterricht', 
                   'Untätigkeit', 'Ankommenden', 'Aufgaben', 'Fremden', 'mitgelöscht', 'Bewältigung', 'Gründen', 'Rauswurf', 'Institute', 'Ausgaben', 'Ministerium', 'Dutzende', 'Übergängen', 'Kleider', 'Präsenz', 'Einrichtung', 'Hooligan', 'Anschlägen', 'Somalier', 'Empathie', 'Freundin', 'Regierungen', 'Haltung', 'Vorschläge', 'Beschleunigung', 'Internationale', 'Immobilien', 'Schmierereien', 'Scheiben', 'Mediziner', 'Terroristen', 'Rücktritt', 'Delegierten', 'Dramaturgie', 'Brüskierung', 'Vorsitzenden', 'Eskalation', 'Übergriffen', 'Belastbarkeit', 'Quantität', 'Qualität', 'Zuwendung', 'Verbündeten', 'Konzepte', 'Verhältnis', 'Strategie', 'Interview', 'Flexibilität', 'Gründlichkeit', 'Institute', 'Befürchtung', 'Vorstellung', 'Schiffe', 'Entgleisungen', 'BIGALKE', 'Hundertschaften', 'Dramatik', 'Ereignisse', 'Einladung', 'Erwartungen', 'Inneren', 'Ultimatum', 'Unterfangen', 'Verordnungen', 'Rückzug', 'Bevölkerung', 'Gemeinde', 'funktionieren', 'Nötigsten', 'Schärfer', 'Vergangenheit', 'Studium', 'Konzert', 'Täuschung', 'Bescheid', 'Einschätzung', 'Beteiligten', 'Möglichstes', 'Belastung', 'Gefahren', 'Unwettern', 'Bildung', 'Bereiche', 'Anstand', 'Kredite', 'Zuwanderern', 'Entschlossen', 'Rückschritten', 'Rückschritt', 'Beitritt', 'Konkurrenten', 'Alternativen', 'Tiefschwarz', 'Kubaner', 'Erwartungen', 'Eingang', 'Religion', 'Angaben', 'Anhänger', 'Vorhaben', 'Ökonomen', 'Schätzung', 'Zusammenhang', 'Auftrags', 'Bekämpfung', 'Albanern', 'Kontakte', 'Regelung', 'Hoteliers', 'Republikaner', 'Kontroversen', 'Reaktion', 'Weichen', 'Schritte', 'Überlastung', 'Termine', 'Ergebnissen', 'Moscheen', 'Vermieter', 'Lässigkeit', 'Umwandlung', 'Frieden', 'Kardinal', 'Illusionen', 'Temperaturen', 'Staates', 'Anreize', 'Leistungen', 'Änderung', 'Position', 'Kriegen', 'Tradition', 'Überfremdung', 'Schlüsse', 'Prävention', 'Hotelier', 'GAMMELIN', 'Überstellung', 'Eritreer', 'Helferinnen', 'Wiederaufnahme', 'Auszüge', 'Durchreise', 'Frachter', 'Grausamkeit', 'Kontingent', 'Begrenzung', 'Durchschnaufen', 'Vorhaben', 'Franzosen', 'Positiven', 'Streichen', 'Einklang', 'Selbstverständlichkeit', 'Umarmung', 'Fähigkeiten', 'Kritiker', 'Feuilletonisten', 'Konsequenz', 'Transporter', 'Steuern', 'Soziales', 'Betrieb', 'Auftakt', 'Strategie', 'Geistliche', 'Einfluss', 'Minderjährigen', 'Unberechtigte', 'Konzerne', 'WERNICKE',
                   'Trennung', 'Schäbig', 'Koordination', 'Terrorismus', 'Feilschen', 'Schärfste', 'Touristen', 'Vorfeld', 'Andrangs', 'Männern', 'Architekten', 'Attentätern', 'Zitaten', 'Produkte', 'Schaden', 'Wertschätzung', 'Bündnis', 'Zuwendungen', 'Einsicht', 'Standorten', 'Regierenden', 'Unterscheidungen', 'Zustimmung', 'Begriffe', 'Verstimmung', 'Entrinnen', 'Gesellschaften', 'Risiken', 'Stiftung', 'Unterstützer', 'Einwanderinnen', 'seiDeutschland', 'Haushalt', 'Amerikaner', 'Reformen', 'Ordnung', 'Quartier', 'Veränderung', 'Erwägung', 'Partnerschaft', 'Trennung', 'Untersuchung', 'Unterschiede', 'Überraschendes', 'Antlitz', 'Widersprüche', 'Abstimmung', 'Mädchen', 'Verstößen', 'Defizite', 'Internets', 'Schleppern', 'Einzige', 'Koordination', 'Betreten', 'Zögerlichkeit', 'Exklaven', 'Jugendlichen', 'Dokument', 'Beziehungen', 'Einschränkungen', 'Zustimmung', 'Überlegungen', 'Unmengen', 'November', 'Oktober', 'Beschränkungen',
                   'Urkunden', 'Italiener', 'Pädagogen', 'Gerüchten', 'Fahnder', 'Darlehen', '17-Jährigen', 'Helfern', 'Überprüfungen', 'Komplex', 'Übernachtung', 'Spediteur', 'Erlebnissen', 'Harmonie', 'Wählern', 'Fremdenfeindlichkeit', 'Anwältin', 'Arbeitslosigkeit', 'Treffens', 'Kommentare', 'Aktivitäten', 'Verlauf', 'Pauschalisierung', 'Gründer', 'Nächten', 'Zerfall', 'Kontinent', 'Sanktionen', 'Obdachlose', 'Aggressionen', 'Zehntausender', 'Gutachter', 'Antwort', 'Abkommens', 'Pfarrei', 'Predigt', 'Projekte', 'Übersetzer', 'Reduzierung', 'Privatsphäre', 'Zustand', 'Teilnehmer', 'Gründungen', 'Erstarken', 'Entwurf', 'Äußerung', 'Schlepperei', 'Verfassung', 'Gutachten', 'Gutachtens', 'Einheimische', 'Pendler', 'Verhältnisse', 'Rückschlag', 'Bearbeitung', 'Bombardierung', 'Vorschriften', 'Freilassung', 'Algerier', 'Fordern', 'Schließung', 'Anfragen', 'Fernseher', 'Europäern', 'Gemeinsamkeiten', 'Formulierungen', 'Migrant', 'Ausweise', 'Reisende', 'Übergreifen', 'Scheitern', 'Freiburger', 'Brüllende', 'Umsetzung', 'Internet', 'Zerbrechen', 'Landung', 'Kämpfer', 'Ärmsten', 'Festlegung', 'Verbleib', 'Aussagen', 'Verwandte', 'Iranern', 'Prämien', 'Kollege', 'Abweichungen', 'Tourismus', 'Fliehenden', 'Legitimität', 'Auswahl', 'Zugeständnis', 'Einheimischen', 'Existenz', 'Andrang', 'Hunderttausender', 'Nervosität', 'Niederlage', 'Kundgebung', 'Putschisten', 'Meinung', 'Nüchternheit', '38-Jähriger', 'Verbleib', 'Umsetzung', 'Unstimmigkeiten', 'Abschaffung', 'Vertraute', 'Nordafrikanern', 'Rentner', 'Gerichten', 'Vorrang', 'Existenz', 'Ventimiglia', 'Vorfällen', 'Szenarien', 'Entscheide', 'Juristen', 'Legalität', 'Jugendliche', 'Bezahlung', 'Gerichten', 'Neunzigern', 'Aussage', 'Libyen-Italien', 'Kandidatur', 'Gipfels', 'Einwohner', 'Spender', 'Dokumente', 'Koalitionäre', 'Computer', 'Algerier', 'Regisseur', 'Beweise', 'Geduldete', 'Projekte', 'Architekt', 'Fördern'
                   'Logistik', 'Manager', 'Programme', 'Führung', 'Erfassung', 'Geschenk', 'Dimension', 'Viertelmillion', 'Straßen', 'Straßen', 'Verbrechern', 'Ungewissheit', 'Beobachtungen', 'Umgebung', 'Projekten', 'sz.de/vr', 'Kanzlerschaft', 'Gastronomie', 'Hemmnissen', 'Zentren', '13-Jährigen', 'Freiwilligen', 'Ertrinken', 'Moslems', 'Studenten', 'Angeboten', 'Interesse', 'Öffentlichkeit', 'Gastgeber', 'Apokalypse', 'Warnungen', 'Ankündigung', 'Schärfe', 'Ressentiments', 'Einkommen', 'Sinfoniker', 'Genozid', 'Armeniern', 'Anhörungen', 'Attesten', 'Zurückhaltung', 'Verschärfungen', 'Nordafrikaner', 'Aushilfen', 'Überprüfung', 'Anstrengung', 'Einbrecher', 'Gespenster'
                   'Zurückweisung', 'ElfJährigen', 'Getümmel', 'Gerichte', 'Sheriff', 'Umsiedlung', 'Dichtung', 'Nationalität', 'Durchsuchungen', 'Venezolaner', 'Einfahrt', 'Zusagen', 'Übernahme', 'Vorgehen', 'Annahme', 'Beistand', 'Aquarius', '31-Jährigen', 'Relevanz', 'Abwanderung', 'Lehrerin', 'Paradigmen', 'Eintracht', 'Pfleger', 'Gezeter', 'Gewürge', 'Beurteilung', 'Explosion', 'Islamistischer', 'Entschleunigung', 'Erklärung', 'Telefonat', 'Demonstrationen', 'Entstehen', 'Tsianos', 'Verwendung', 'Verkehr', 'Meldungen', 'Beifang', 'Phänomen', 'Ausschiffen', 'Befehle', 'Allianzen', 'Abschottung', 'Aussprache', 'Egoisten', 'Bescheide', 'Weigerung', 'Aggression', 'Abreise', 'Angebote', 'Irakers', 'Liberalen', 'Fliegern', 'Fasanerie', 'Voßkuhle', 'Journalisten', 'Schwangere', 'Vorgehen', 'Rohingya', 'Normalität', 'Eindruck', 'Rückgang', 'Aufsicht', 'Weiterbildung', 'Demonstrationen', 'Überforderung', 'Wackeliger', 'Offenheit', 'Hinweis', 'Dialekt', 'Inflation', 'Erosion', 'Tendenz', 'Verleugnung', 'Scharfmacherei', 'Gambiern', 'Bereicherung', 'Längerem', 'Erneuerung', 'Richtungen', 'Richtigen', 'Begleiter', 'Odyssee', 'Anläufen', 'Ungewissen', 'Knochen', 'Röntgen', 'Schwindler', 'Ernüchterung', 'Unrecht', 'Anwesenheit', 'Exempel', 'Rechtsradikalen', 'Rechtsradikaler', 'Rechtsradikale', 'klarzumachen', 'Bestürzung', 'Zurückweisungen', 'Papiers', 'Eingriff', 'Auswertung', 'Abgrund', 'Besuche', 'Beispiele', 'Freispruch', 'Verloren', 'Dickicht', 'Restaurant', 'Abgaben', 'Fehlers', 'Prioritäten', 'Maxvorstadt', 'Vorgehen', 'Populismus', 'Vorbereitungen', 'Offensive', 'Durcheinander', 'Ungereimtheiten', 'Sprücheklopfer', 'Nüchtern', 'Verstöße', 'Angstmacher', 'Beschuldigter', 'Enteignung', 'Schmuggler', 'Kürzungen', 'Uiguren', 'Bereicherung', 'Beratung', 'Zurückweisung', 'Verschärfen', 'Nachzug', 'Analyse', 'Verlagerung', 'Tendenz', 'Tätigkeit', 'Betrieben', 'Abmachungen', 'Fixierung', 'Stillstand', 'Stillstands', 'Rechtsextremer', 'Gemeinsamkeit', 'Katalog', 'Abschottung', 'Begeisterung', 'Wegweisend', 'Algorithmus', 'Hundertausende', 'Ertrunkene', 'aufnahmen', 'Ausgesperrt', 'Taschen', 'Produkten', 'Aufhalten', 'Diktator', 'Zufluchtslose', 'Ankommen', 'Förderung', 'Institutionen', 'Schippern', 'Begriff', 'Veranstaltung', 'Aufgestauter', 'Tischler', 'Denkmal', 'Sondierungen', 'Frappierende', 'Schwurbeln', 'Abweichung', 'Glaubwürdigkeit', 'Engagment', 'Wurzeln', 'Engagierte', 'Disruptiven', 'Entschädigung', 'Nasibullah', 'Bilanzen', 'Zuhause', 'Überleben', 'Pranger', 'Falschen', 'Revolution', 'Kapazität', 'Jammertal', 'Kontrolleure', 'Bescheide', 'Begrenzten', 'Diplomat', 'Grundsätzliches', 'Verfahren', 'Verfahrens', 'Zurückweisung', 'Zurückweisungen', 'Offensive', 'Jüngsten', 'Tatenlosigkeit', 'Kürzung', 'Bescheide', 'Leiterin', '19-Jähriger'
                  , 'Wahrheit', 'Wahrheiten', 'Student', 'Studentin', 'Studenten', 'Strässer', 'Durchbruch', 'Gerhart-Hauptmann-Schule', 'Vermittlung', 'Kommentar', 'Verhinderung', 'Rumänen', 'Änderung', 'Änderungen', 'RESPEKT', 'Uniform', 'Eröffnung', 'Stöckelplatz', 'Infizierter', 'Zimmern', 'Senator', 'Kathedrale', 'Asylantin', 'Australier', 'Ausblick', 'Einstellung', 'Johannisgasse', 'Kundgebungen', 'Minderjähriger', 'Vermummte', 'Leipzig-Schönefeld', 'Versuchter', '13-Jähriger', 'Antragsteller', 'Ausstieg', 'Gelassenheit', 'Hauptmann-Schule', 'Gymnasium', 'Erfolge', 'Sebastio', 'Schriftsteller', 'Hauptschüler', 'Akademiker', 'Überfüllung', 'Dezernentin', 'Richterin', 'Aufsatz', 'KANZLERIN', 'Asylanten', 'Äußerungen', 'Paradies', 'Hohehorst', 'Liebermann-Straße', 'Reportage', 'Minderjähriger', 'Betreibern', 'Bleiben', 'Ohlauer', 'Verzögerung', 'Fertigstellung', 'Autonome', 'Besetzer', 'Spenden', 'Aznavour', 'Ausrüstung', 'A-Platz', 'O-Platz', 'Gerhardt-Hauptmann-Schule', 'Gelassenheit', 'Verfolgte', 'Bischofswerda', 'Schwerverletze', 'Senator', 'dutzend', 'Festung', 'Berlinerin', 'Protestler', 'Farmen', 'Chaoten', 'Gekritzel', 'Drängen', 'Besetzer', 'Senatorin', 'Entsetzen', 'Dienste', 'Entführer', 'Kleinod', 'Misshandelter', 'Aufmarsch', 'Plakaten',
                  'ENDLICH', 'Teilnehmern', 'Institut', 'Vermisster', 'Unfassbarer', 'Versicherungen', 'Verträge', 'Übertreibung', 'Bewohnerin', 'Spaltung', 'Syrerin', 'Truckern', 'live@night', 'Milliardär', 'Vereine', 'Mächtigster', 'Ezadeen', 'KNALLHARTER', 'Kleidung', 'Kondome', 'Patriot', 'Ansporn', 'Verwüstungen', 'Zusammenbruch', 'Aufmachen', 'POLITIK', 'Gewissen', 'Handeln', 'Transportern', 'Historiker', 'Somalierin', 'Gerechtere', 'Aufteilung', 'Schicksalen', 'Albaner', 'Brandstift', 'UNTERSTÜTZUNG', 'GESCHÄFT', 'Gesprächen', 'Hetzerin', 'Nationalist', 'HOFFNUNG', 'Auskunft', 'DEUTSCH', 'Anwohnerin', 'Fackeln', 'Parolen', 'Wartenden', 'Fregatte', 'FLÜCHTLINGE', 'Rauchenecker', 'Schlacht', 'Tumulte', 'Einreisen', 'ZEICHNET', 'Tohuwabu', 'Olbricht-Kaserne', 'Bayernkaserne', 'Umsiedlung', 'Klamotten', 'Flüchten', 'Asoziale', 'Sophienterrasse', 'Standfestigkeit', 'Handlungsfähigkeit', 'Ehreamtler', 'Schiffbrüchige', 'Duschen', 'Mietern', 'Verbrechens', 'Obdachloser', 'Vermisstes', 'Angreifer', 'Nikolaus', 'Plauener', 'Intervention', 'Theater', 'Passanten', 'Ertuğrul', 'Brühlpelz', 'Schnackenburgallee', 'Ayvacik', 'Fasching', 'Razzien', 'Wissenschaft', 'Ehrlichkeit', 'Ergebnisse', 'Islamisierung', 'Beamten', 'Despoten', 'Saarstollen', 'Gefängnisse', 'Umgangs', 'Gesprächen', 'Klatsche', 'Fischer', 'Tumulte', 'Journalistin', 'GRÖSSTEN', 'Organisatoren', 'Kontingenten', 'Knackis', 'Illegale', 'Trucker', 'Trikots', 'Schloss', 'Trommeln', 'Exklusiv', 'Überfall'
                  ,'Schlägen', 'Gewerbe', 'Bezirken', 'Momente', 'Finanzierung', 'Klüngel', 'Schönheit', 'Bescherung', 'Kollegen', 'Junkies', 'Pennern', 'Pöbeln', 'Streife', 'Viertel', 'Bildchen', 'Ertrunkener', 'HelpTo.de', 'Nachbar', 'Beamter', 'Tuberkulose', 'Polizistin', 'Auseinandersetzung', 'Auseinandersetzungen', 'Velodrom', 'Unterschriften', 'Connewitzer', 'Aufregung', 'Dölziger', 'Anblick', 'Durchsage', 'Mikrofon', 'Fotograf', 'Schelztorhalle', 'Pappritzer', 'Komplizen', 'Bürgerschaft', 'Lemsahl-Mellingstedt', 'Tochter', 'Herberge', 'Neugraben', 'Hingucker', 'Flüchtlinge', 'Königsplatz', 'Reporter', 'Protesten', 'Rapport', 'Pilzesuchen', 'Patienten', 'Asozialer', 'Zynismus', 'Arabischsprachige', 'Abrechnung', 'Anwohnern', 'Zollenspieker', 'Schnaps', 'Smartphone', 'Software', 'Ausstand', 'Erzieherinnen', 'Neugrabener', 'Dezember', 'Verhandlungen', 'Kapitän', 'WAHRHEIT', '20-Jähriger', 'Bliesdalheim', 'Akçakale', 'Dresden-Klotzsche', 'Paletten', 'Asylant', 'Wieviel', 'Sprüchen', 'Wiedereinführung', 'Quarantäne', 'Shitstorm', 'Vielfaches', 'Stöcken', 'Flaschen', 'Verletzter', 'Aufregen', 'Talkshow', 'Fassaden',
                  'Aufregen', 'Auftritt', 'Aushang', 'Erinnerungen', 'Rot-Grün', 'Schämen', 'Ballons', 'Bildern', 'Besoffener', 'Frühchen', 'Gruppen', 'Verschleierung', 'Einführung', 'Anwälte', 'Hilfsbedürftige', 'Entgleisung', 'merkelstreichelt', 'Verkäuferin', 'Gemüter', 'Pavillon', 'Radikalisierung', 'Wiederverinigung', 'Vierjähriger', 'Schnupfen', 'Heiserkeit', 'Rassist', 'Schauspieler', 'Gebäude', 'Schrecken', 'Download', 'Kongress', 'Reagieren', 'Ernst-Grube-Halle', 'Sozialdemokratische', 'Verwirrung', 'Kleiner Poststraße', '26-Jähriger', 'Grube-Halle', 'Aktionen', 'Genehmigung', 'Unwetter', 'Bezirke', 'Besetzung', 'Zuständen'
                  , '17-Jähriger', 'Harttung', 'Investor', 'Silvester', 'Flüchtlinge', 'Willy-Brandt-Haus', 'Kindern', 'Enthauptung', 'Grillen', '-Städte', 'Schläger', 'Schlappe', 'Makulatur', 'Brandstiftungen', 'Entschärfung', 'Scheunen', 'Flächen', 'Gestoppte', 'Aufnahmen', 'Stellung', 'Spielen', 'Bambule', 'Vermummter', 'Maklers', 'Transit', 'Bestattung', 'Heuchelei', 'Erschreckender', 'Heidenaus', 'Bordell', 'Behauptungen', 'Maklers', 'Reservisten', 'Grauens', 'Zufahrt', 'Senatoren', 'Interview', 'Stadion', 'Berlin-Wittenau', 'Planung', 'Experte', 'Transit', 'Platten', 'Pöbeleien', 'Johanniter', 'Jehovas', 'Bund+++', 'Protokoll', 'Dörverden', 'Betrunkner', 'Pullover', 'SoGehtEs', 'Gedanken', 'Skateboards', 'Kindern', 'Alfred-Faust-Straße',
                  'Attackierter', 'Besserung', 'WETTSCHLIESSEN', 'Schlächter', 'Umwandlungen', '39-Jährigen', 'Professor', 'GEPLATZTER', 'Platzen', 'Tschetschenen', 'REGIERUNG', 'Comeback', 'Konkurrenz', 'Versäumnis', 'Besonderheit', 'ZÜNDETEN', 'PRÜGELN', 'Selbstbestimmung', '70.Jubiläum', 'Polemik', 'Schützen', 'Amadeu-Antonio-Stiftung', 'Kantinen', 'SPIELEN', 'MILLIARDEN', 'Zündelnder', 'Schexit', 'Vertrag', 'Festerling', 'Heiligsprechung', 'Materialien', 'Verwarnung', 'ARBEITEN', 'Karnevalisten', 'Schlepperin', 'Ramadan', 'Unterkünft', 'Auszubildenden', 'Seminar', 'Verband', 'Bundesweite', 'Zeitungen', 'Abriegelung', 'Vorsitz', 'Lächeln', 'Wundercover', 'Nudisten', 'Nord-Afrikaner', 'Johannstadt', 'Zeichnungen', 'Helferin', 'Vergewaltiger', 'Karneval', 'Selbstbestimmung', 'RETTUNG', 'Gebüsch', 'Kassierten', 'PROBLEM', 'Stühlen', 'Verhöhnung', 'Gitarre', 'FLÜCHTLING', 'Bewachung', 'Gespräche', 'Fesseln', 'ABSCHIEBUNG', 'Abstrichen', 'Verband', 'DEUTSCHES', 'Entstehung', 'Klopperei', 'Hostels', 'Egomane', 'Komplott', 'Freitalerin', 'VERLETZT', 'Städten', 'Schikanierten', 'Tunesiern', 'Ausgedachter', 'Allerkleinstes', 'Pistorius', 'chtlinge', 'Countdown', 'LETZTEN', 'STUNDEN', 'Schranken', 'Grenzer', '22-Jährigen', 'Asselborns', 'Gequassel',
                  'Festival', 'Beleidigung', 'Angeklagte', 'Tomaten', 'Leichterer', 'Verschluss', 'Vorsitzender', 'Empfänger', 'Appellen', 'Verurteilung', 'Hotspot', 'Portugiese', 'Geschenke', 'Angeworben', 'Dönhoff-Preis', 'Erfindung', 'Ungeziefer', 'Ausreden', 'Sterben', 'Scanner', 'Inferno', 'Freundlichkeit', '34-Jähriger', 'Torheit', 'Tschechen', '18-Jähriger', 'Bekannte', 'Öffnung', 'Universität', 'Hintern', 'Studentinnen', 'Gillamoos', 'Versehen', 'Formular', '16-Jährigen', 'Parlamente', 'Dämpfer', 'Kommuniqué', '38-Jährigen', 'internationaler', 'Stärkung', 'Nachbesserungen', 'Vorbehalt', '31-Jähriger', 'Ausschluss', 'Photobomb', 'Aufhebung', 'Grabscher', 'Beschäftigung', 'Plädoyers', 'Unterbringungen', 'Picknick', 'Scheich', 'Agenturen', 'Beratungen', 'Aufwendungen', 'Gambier', 'Bautzener', 'Sportler', 'Blankense', 'Protestanten', 'Erzbistum', 'Überschuss', 'Streits', 'Soziologin', 'Februar', 'Training', 'Fälschung', 'ratzfatz', 'Partnern', 'Taskforce', 'Fortbestand', 'Intellektuelle', 'Schaffung', 'Alphabetisierung', 'Verdachts', 'Tagelanger', 'Versteck', 'Widerstands', 'Neuntklässler', 'Jugendlicher', 'Straffällige', 'Popularität', 'Dynamik', 'Tourist', 'Abbruch', 'Abgeordnete', 'Belästigungen', 'Wasserbetrieben', 'Gemisch', 'Akzeptanz', 'Auszeichnung', 'Dolmetscherin', 'Rechtsprechung', 'Aufruhr', 'Vatikan', 'Busfahren', 'Rush-Hour', 'Sprecher', 'Bengalis', 'Wünschen', 'Schickten', 'Vorangehen', 'Verordnung', 'Bedeutung', 'Wandern', 'Trebesius', 'Vorsitzende', 'Chinesen', 'Bündnisse', 'Ehrenamtlichen', 'Humanist', 'Realist', 'Schweden', 'Sanierung', 'Neuerungen',
                  'Streich', 'Glietsch', 'Plätzen', 'Geflüchteter', 'Gesicht', 'Al-Bakr', 'undercover', 'Zuschlag', 'Nobistor', 'Ausgleich', 'Basepohl', 'Klein-Borstel', 'Ersticken', '33-Jährigen', 'Anleitung', 'Polygamie', 'Beobachtung', 'Antänzern', 'Verbrecher', 'Kentern', 'Scheuer', 'Durchleuchtung', 'Deutschlernen', 'Buddenbrookhaus', 'Erkrankungen', 'Zehntel', 'Komplize', 'Hamburger', 'Entgegenkommen', '21-Jähriger', 'Diakonie', 'Sticheleien', 'Angeklagter', 'Gleichberechtigung', 'Zufriedenheit', 'Verunglimpfung', 'Sachlichkeit', 'Gewicht', 'Königshain-Wiederau', 'Watschen', 'Verlegung', 'Startklar', 'Entlassungen', 'Verdienst', 'Sprünge', 'Zerstrittene', 'Tornados', 'Symphoniker', 'Symphonie', 'Demografie', 'Zusammenhalt', 'Zigarette', 'Auslöser', 'Brandes', 'Eignung', 'Horstismus', '21-Jährigen', 'Twitter', 'Gesuchter', 'Zigaretten', 'Attentäters', 'Fremdenfeindliche', 'Bedürftige', 'Revolten', 'Stacheln', 'Alt-Sachsenhausen', 'Einblicke', 'Ehrenamtler', 'Fraktionen', 'Nachahmung', 'Bewegung', 'Krankheiten', 'Leiters', 'Austritt', 'Grammatik', 'Ingenieur', 'Containern', 'Wohnungslose', 'Scheidung', 'Garantie', 'Sachlichkeit', 'Beteiligung', '|Anstifter', 'Nutella', 'Überwältigung', 'Al-Bakrs', 'Verletzten', 'Ausgabe', 'Klempner', 'Früchtchen', 'Dschaber', 'Steigerung', 'Verheiratete', 'Verurteilte', 'Milliaden', 'Wiedervereinigung', 'Rückhalt', 'Expertin', 'Salafismus', 'Überforderte', 'Zusammenhalt', 'Unterstützern', 'Eingliederung', 'Befugnisse', 'Verräter', 'Gremien', '16-Jähriger', 'Kommando', 'Klein-Borstel', 'Eingliederung', 'hunderte', 'Methoden', 'Religionen', 'Beendigung', 'Jordanischer', 'Angriffen', 'Überschwemmungen', 'Nationalitäten', 'Qualifikation', 'Durchleuchtung', 'Bränden', 'Instrumente'
                  , 'Widerlicher', 'Hindernisse', 'Merkel', 'Durchwinken', 'Unmündigkeit', 'Urteile', 'Strömung', 'Politologe', 'Schwimmerin', 'Finanzen', 'Auswirkungen', 'Fälschungen', 'Impfungen', 'Afrikanerin', 'Margitte', 'Rassisten', 'Schiffen', 'Strobls', 'Wahrsager', 'Interessenten', 'Präsidium', 'Senegalesen', 'Widerspruch', 'Abstand', 'Nachfrage', 'Hindernisse', '24-Jähriger', 'Premiere', 'Publikum', 'Belästigung', 'Auszeichnungen', 'Schwerverletzte', 'Rückreisen', 'Aufgespießter', 'Grimm-Benne', 'Kriminellen', 'Herrnhuter', 'Benachteiligung', 'Bäckerei', 'Freistellung', 'Unwesen', 'Einbußen', 'Schlamperei', 'Turnier', 'Besitzes', 'Marienplatz', 'Frösche', '26-Jährige', 'Historikerin', 'Häufung', 'Erlaubnis', 'Ausflug', 'Säuberung', 'Desertierter', 'Nationalismus', 'Talente', 'Ausschreibungen', 'Zusammenstößen', 'Schmarotzer', 'Menschheit', 'Millionär', 'Randalierer', 'Verkettungen', 'Anwerbung', 'Radikale', 'Hindernisse', 'Posaunist', 'Provider', 'Befragte', 'Prüfungen', 'Identitäten', 'Spannungen', 'Partner', 'Math4Refugees', 'Beethoven', 'Allianz',
                  'Bemühen', 'Propaganda', 'Schilderungen', 'Breitscheidplatz', 'Machete', 'Qualifizierung', 'Wiederholung', 'Schwert', 'Drohnen', '14-Jähriger', 'Verhandlung', 'Gewerkschaften', 'Präfekten', 'Chronologie', 'Ansehen', 'Botschafterin', 'Schönreden', '35-Jähriger', 'Schilder', 'Grün-Schwarz', 'Explosionen', 'Mentoren', 'Begleitung', 'Gerstungen', 'Behinderte', 'Dresdnern', 'Trauern', 'Bulldozer', 'Schülerin', 'Flüchtline', 'Bürgern', 'Kammern', 'Bremerin', 'Einlenkens', 'Kristoph', 'Fördern', 'Zuschuss', 'Schmieden', 'Einlenken', 'Behandlungen', 'Bewaffnetem', 'Heilige', 'Psychologe', 'Schuldfähigkeit', 'Regierende', 'Gladiator', 'Verhandlung', 'Anzeichen', 'Vorlage', 'Vermisst', 'Palästinenserin', 'Afghanin', 'Meister', 'Mosaike', 'Korrekturen', 'Strenge', 'Suspendierung', 'Max-Braun-Zentrum', 'Bedürfnisse', 'Senioren', 'Teenager', 'Regierende', 'Anweisungen', 'Meldung', 'Bischöfen', 'Vergewaltigern', 'Scheibe', 'Rentnerin', 'Differenzen', 'Philosoph', 'Immunität', 'Bistums', 'Objekte', 'Fressen', 'Zuweisungen', 'Material', 'Schreie', 'Bausbacks', 'Ängsten', 'Quartett', 'Präfektin', 'Hetzten', 'Parsifal', 'Zählung', 'Identifizierung', 'Björsenweg', 'Blankenese', 'Jesuitische', 'Liedern', 'Vahrenheide', 'Aufkündigung', 'Explosionen', 'Bibliotheken', 'Parzelle', 'Lichtenhagen', 'Vertreibung', 'Skandale', 'Betreibers', 'Schüssen', 'Sudanese', 'Staffele', 'Graffiti', 'Gewerkschaften', 'Johlige', 'Malteser',
                  'Blauäugigkeit', 'Englisch', 'Rauschhaftes', 'Erwachsene', 'Betrunkener', 'Polster', 'Maritim', 'Zuschüsse', 'Krematorium', 'Kompass', 'Göring-Eckart', 'Unterkühlung', 'Wörtern', 'Wichtigkeit', 'Gelände', 'Kurdisch', 'Showdown', 'Weitergabe', 'Erwerbslose', 'Terrors', 'Bengalos', 'Vermögen', 'Website', 'Voraussetzung', 'Vierlingen', 'Mädchens', 'Kaserne', 'Johannes-Stiftung', 'Schleusungen', 'Besucher', 'Humanitärer', '32-Jähriger', 'Zwinger', 'Stigmatisierung', '36-Jähriger', 'Verbindungen', 'Rüpeleien', 'Verbote', 'Lockerungen', 'Ghettos', 'Mäßigung', 'schaffen', 'Rheinland-Pfälzer', 'Gerettete', 'Erwachsene', 'Untiefe', 'Beauftragter', 'Ausspruch', 'Impulse', 'Spannung', 'Bestehen', 'Radikaler', 'Scheeres', 'Bedrohungen', 'Psychiater', 'Korrupte', '22-Jähriger'
                  ,'Brötchen', 'Fehlern', 'Abstriche', 'Fernbleiben', 'Variante', 'Pfötchen', 'Gleichgültigkeit', 'MIGRANTEN', 'GEFÄLSCHTEN', 'Vergebung', 'Grübeln', 'Geständnisse', 'Angriffs', 'Berater(+', 'Joggern', 'Hinrichtung', 'Kandidatin', 'Lockerung', 'MOHAMMAD', 'Konvertiten', 'Verfolgter', 'Internierung', 'Kartoffel', 'Generälen', 'Admiralen', 'EXKLUSIV', 'Angeklagten', 'Fortbildung', 'EMPÖRUNG', 'Aufklären', 'Erinnerung', 'Tristesse', 'Fünfjähriger', 'TERRORISTEN', 'Rot-grüne', 'Grinsen', '27-Jähriger', 'Schwester', 'Integrationswilligen', 'anglager', 'REGULÄREN', 'Rontgen', 'Berufung', 'Töchter', 'Hamburg-Eidelstedt', 'Verdächtigem', 'Zusammenlebens', 'Ausreisepflichtige', 'Lebenslang', 'Abzocker', 'MILLIONEN', '26-Jährigen', 'Dankbarkeit', 'Petitionen', 'Main-Kinzig-Kreis', 'Ausreisepflichtigen', 'OrbánÄußerungen', 'Soopikas', 'VERLIEREN', 'Insolvenz', 'MEINUNG', 'Durchsuchung', 'Psychologen', 'Station', 'Andacht', 'Einsichtnahme', 'Kandidaten', 'Freigabe', 'Abmeldung', 'Munition', 'Kompetenzen', 'Eindämmung', 'Klavier', 'Emirate', 'Predigern', 'Lohengrin', 'Abzocker', 'Grenz-Öff', 'EISERNEN', 'AFGHANEN', 'TAUCHTEN', 'Abgelehnter', 'Ungarn-Orbán', 'Fantastischen', 'Beginner', 'Fünftel', 'Viadrina-Preis', 'Verkauf',
                   'Moderatorin', 'Inhaftierung', 'Baldiger', 'Lene-Voigt-Schule', 'Schwerverletzter', 'Abzocke', 'Freundes', '41-Jähriger', 'Töchtern', 'Nigerianer', 'Barrieren', 'Syrischer', 'Promille', 'Matratze', 'Insassen', 'Al-Abadi', 'Lesenlernen', 'Aushöhlung', 'Camperin', 'Hangars', 'Cholera', 'Fotografen', 'Sanktion', 'Bamberger', 'Reichen', 'Zwölfjähriger', 'Rentnern', 'Vorfall', 'Vorfälle', '27-Jährigen', 'Auszeichnunge', 'Schwung', 'Geflüchtetn', 'Höhler-Biennale', 'Defizit', 'Defizite', 'Dolmetschern', 'Vereinfachter', 'Konsequent', 'Registrierungen', 'Burladinger', 'Unterlage', 'Unterlagen', 'Industrie', 'Freundes', 'JA-maika', 'NEIN-maika', 'Politologen', '20-Jährigen', 'Zwölfjähriger', 'TRUMPine', 'Hertie-Stiftung', 'Organisationen', 'Versteigerung', 'Abrüstung', 'Homosexuellen', 'Irakischer', 'Agentur', 'Toilette', 'Bretter', 'Beilegung', 'Nigerianer', 'Uraufführung', 'Leitung', 'Absolventen', 'Staubsauger', 'Nachahmer', 'Erkältung', 'Broschüre', 'Übernachtungen', 'Veedelszoch', 'Irakern', 'Vergrößerung', 'Respekts', 'Einkauf', 'Streichung', 'Management', 'Fortsetzung', 'Verbänden', 'Leidende', 'Inobhutnahmen', 'Wehrpflichtige', 'Hungernde', 'Vorbereitung', 'Praktikanten', 'Journalist', 'Manöver', 'Manipulation', 'Alexander-Otto-Stiftung', 'Beisetzung'
                  , 'Schreiber', 'Therapeuten', 'Härterer', 'Spitzen', 'Verhärtete', 'Debakel', 'Mauschelei', 'Einschleusung', 'Sepp-Herberger-Urkunde', 'Kameradschaft', 'Duvenacker', 'Makaber', 'Islamist', 'Bescheiden', 'Frankfurter', 'Hamburg-Hammerbrook', 'Bewaffneten', 'Eintritt', 'Vermietung', 'Sorgfalt', 'Vertretern', 'Suizid', 'Bewohnern', 'zehntausende', 'Tragödien', 'Rettern', 'Abnehmer', 'Matratzen', 'Sterntalerpreis', 'Enkelin', 'Trumpine', 'Gefängnissen', 'Monstranz', 'hergetragen', 'Leistung', 'Verstoß', 'Vorschrift', 'Bürgerrechtler', 'Rückkehrern'
                  , 'Bemerode', 'Untreue', 'Entgeht', 'Ignorierte', 'Klopfen', 'Trumpeten', 'ANTWORTEN', 'Stehhofer', 'Drehhofer', 'Urlauber', 'Verbrechers', 'Allasane', 'Prothesen', 'Tauziehen', 'Pakistani', 'WOLLTEN', 'Mexikaner', 'Beobachtern', 'Lifeline', 'ANGRIFFE', 'Mudasar', 'Aufenthalt', 'Fössebad', 'Betrunkene', 'Zurückweisen', 'BEKOMMEN', 'VERURTEILTER', 'KRIMINALITÄT', 'Grapscher', 'Wirrwarr', 'Abgeschobener', 'Veruntreuung', 'Gittern', 'Piloten', 'Islamismus', 'Beliebtheit', 'rgermeister', 'Intoleranten', 'Steinen', 'Superhorst', 'QUA-Lition', 'Plakate', 'Whirlpool', 'Killers', 'PAKISTAN', 'Anschuldigungen', 'Klischees', 'LEISTEN', 'Knästen', 'HILLARY', 'Letzter', 'Bestochen', 'Westens', 'Vertrauter', 'Pakistani', 'Vorgänger', 'SCHLIMMSTEN', 'Gewahrsam', 'Piloten', 'Mexiko-USA', 'KOMMENTAR', 'WILLIGEN', 'Präsidentin', 'Vertretung', 'Mauscheleien', 'Togolesen', 'Verhaftung', 'Tumulten', 'Kameruner', 'Modezars', 'FDP-Kubicki', 'Ausweitung', 'Auspuff', 'Gefeuerter', 'Hochschwangeren', 'Intoleranten', 'ABSURDESTER', 'Attrappe', 'Geräusche', 'Auergründel', 'Dschihadisten', 'Libyern', 'Anstoßes', 'Urlauber', 'Schwimmerinnen', 'Schöffin', 'Verurteilter', 'Ausdehnung', 'Anlegen', 'Leitungen', 'Zweijährige'
                  ,'Erwartung', 'Schulung', 'Krieges', 'Vorkehrung', 'Weiter-so', 'Ruander', 'Rheingau-Taunus', 'Häuschen', 'Piraten', 'Dekanin', 'Dekanat', 'Birkengewann', 'Bulgaren', 'Hotline', 'Krachender', 'Ratschlag', 'Ratschläge', 'Zuweisung', 'Einstufung', 'Apartments', 'Schlüchtern', 'Moderator', 'Wowereit', 'Dächern', 'Diplomatie', 'Einwandern', 'GLEICHE', 'Dekanat', 'Gülegen', 'Richtige', 'Überlassung', 'Separatisten', 'Söldner', 'Informant', 'Kämpfern', 'Pauschalen', 'Nutzung', 'Klinikum', 'Symbole', 'Spinner', 'Vlaska-Gasse', 'Informationen', 'Dokumentation', 'Aufarbeitung', 'Provisorium', 'Diabetes', 'Reglungen', 'Eingewöhnen', 'Bedenken', 'Gettoisierung', 'abwarten', 'Gefangene', 'Passage', 'Befragung', 'Schweinischer', 'Kapitalismus', 'Verdopplung', 'Effekte', 'Platzes', 'Graves-Kaserne', 'Übergabe', 'Zuweisung', 'Betroffene', 'Beschützer', 'Wegfall', 'Dezernent', 'Option', 'Nutzung', 'Karawane', 'Glaubens', 'Zaudernde', 'Verpflegung', 'Privatisierung', 'Akteure', 'Erschöpfung', 'Aufnehmen', 'Teuerste', 'Einzelheiten', 'Gremium', 'Ideologie', 'Nausikaa', 'Schiffbrüchigen', 'Baracke', 'Selbstbehauptung', 'Einwandern', 'Punkten'
                  , 'Hickhack', 'Schwarzmalerei', 'Schwächung', 'General', 'Operation', 'Vermeidung', 'Souveräne', 'Segregation', 'Führungen', 'Bankern', 'Winters', 'Liebigschule', 'Unbehauste', 'Angelengenheiten', 'Spendabel', 'Täuschungen', 'Erweiterung', 'Information', 'Depressionen', 'Strukturen', 'Profiteure', 'Zulassung', 'Büchern', 'Flipflops', 'Jacob-Grimm-Preis', 'Asserate', 'Transparenz', 'Philologen', 'Vorstand', 'Schuldigkeit', 'Bewunderung', 'Elektriker', 'Element', 'Ansichten', 'Paradoxien', 'Helfens', 'Operation', 'Globalisierung', 'Juvente', 'mögen’s', 'fieberhaft', 'Tacheles', 'Seismograph', 'Knarzen', 'Preußen', 'Entdeckung', 'Verkürzung', 'Untertaunus', 'Ereignis', 'Intoleranz', 'Atmosphäre', 'Stöpsel', 'Umgestaltung', 'Überraschung', 'Jesuiten', 'Kapitale', 'Inklusion', 'Stabilität', 'Betteln', 'schimpfen', 'Peter-Jordan-Schule', 'Unruhen', 'Abgekoppelt', 'Unterbrechung', 'Versammlungen', 'Maulheldentum', 'Dringlichkeit', 'Übersetzungen', 'Unanständigen', 'Verlust', 'Investoren', 'Rechnen', 'Programmierer', 'Rheinhalle', 'Mobilität', 'Befindlichkeiten', 'Wanderung', 'Ressentiment', 'Vorschlägen', 'Dialektik', 'Verbindlichkeit', 'Einstand', 'Bankrott', 'Kommissare', 'Patrouille', 'Schmiere', 'Kelley-Barracks', 'Fluchten', 'Begegnung', 'Zaorálek', 'Berechnungen', 'Hoffest', 'Einfühlung', 'Behinderung', 'Wetteraukreis', 'Magistrat', 'Ausschuss', 'Unangemessen', 'Beschlagnahme', 'Positionierung', 'Verringerung', 'Kritikerin', 'Erweiterung', 'Operation', 'Balkanisch', 'Beschlüssen', 'Rentsch', 'Reparieren', 'Unmöglichkeit', 'Symptome', 'Bemühungen', 'Refugium', 'Pfeiler', 'Konstruktives', 'Wetteraukreis', 'Fluktuation', 'Sportsfield', 'Tablets', 'Raffelhüschen', 'Möglichkeit', 'Stöpsel', 'Flasche', 'Herzlichkeit', 'Brüderlichkeit', 'Metropole', 'Franz-Böhm-Schule', 'Gedenkens', 'Synagoge', 'Erwerbslosen', 'Brüdern', 'Schwestern', 'Unkenntnis', 'Antreten', 'Controllers', 'Sprachlosigkeit', 'Berechnung', 'Defizits', 'Aufgetaucht', 'Fleisches', 'Zwitschern', 'Scherzen', 'Widerstände'
                  ,'Zeugnis', 'Anträgen', 'Polarisierungen', 'Versäumnisse', 'Lindscheid', 'Substanz', 'Quartiere', 'Main-Taunus', 'Verzögerungen', 'Schiene', 'Geplapper', 'Gebieten', 'Vorübergehender', 'Beschuldigungen', 'Lethargokratie', 'Pragmatiker', 'Fertigmachen', 'Aufstieg', 'Vorstellungen', 'Auswanderer', 'Gründung', 'Babysitting', 'Windeln', 'Hasselgrundhalle', 'Verbündeter', 'Laptops', 'Zweifeln', 'Geschwindigkeit', 'Rangeleien', 'Differenzierungen', 'Durchwinken', 'Anrufen', 'Wetterauer', 'Trainer', 'Machbaren', 'Partsch', 'Synchronsprechen', 'Dominanz', 'Verzehr', 'Caritas', 'Bedürftigkeit', 'Imperialismus', 'Vorgaben', 'Vorsorge', 'Eigentum', 'Schmerzen', 'Universitäten', 'Knöllchen', 'Heidenrod-Kemel', 'Definition', 'Gebühren', 'Meinungen', 'Formulare', 'Rendite', 'Nostrum', 'Angelegenheit', 'Unzufriedenheit', 'Anmietung', 'Gebäuden', 'Uniformen', 'Anrechnung', 'Gefecht', 'Aufbruchs', 'Vorsicht', 'Besonnenheit', 'Bequemlichkeit', 'Ansagen', 'Irritation', 'Tropfen', 'Wirksamkeit', 'Emigranten', 'Prognosen', 'Landauf', 'Zerstörung', 'Asiaten', 'Burgwiesenschule', 'Verführung', 'Resolution', 'Sensibilität', 'Verzeichnis', 'Haushalte', 'Spediteure', 'Sperrung', 'Fristen', 'Bekenntnisse', 'Sicherung', 'Steuerung', 'Salzburgern', 'Ursprung', 'Unheils', 'Vereins', 'Private', 'Stiftungen', 'Stiftungen', 'Katalysator', 'Motiven', 'Bekunden', 'Vielfalt', 'Notizen', 'Tickets', 'Boykott', 'Operationen', 'Jubiläum', 'Gymnasien', 'Exulanten', 'Einwände', 'Beethoven-Preis', 'Ann-Dorit', 'Schlimmste', 'Unklarheit', 'Illusion', 'Destabilisierung', 'Volleyball', 'Volleyballs', 'Vorpreschen', 'zurückzerren', 'Methode', 'Zaghafte', 'Abwegigen', 'Willkommens', 'Kleinkariertes', 'Geldermalsen', 'Durchschleusung', 'Erwarte', 'Geldern', 'Geschlecht', 'Techtelmechtel', 'Eltville', 'Unklarheiten', 'Lächelns', 'Anfrage'
                  , 'Lehrern', 'Gefährdung', 'Traumata', 'Dresdner', 'Verkleinerung', 'Geschwister-Scholl-Schule', 'Funktion', 'Empfehlungen', 'Tausenden', 'Gehilfe', 'Buchhügel-Turnhalle', 'Diskothek', 'Faktoren', 'Exotismus', 'Gestalt', 'Beschäftigten', 'Bandagen', 'Lotterie', 'Unordnung', 'Triumph', 'Behälter', 'Schnabel', 'Weiterverhandeln', 'Prominenz', 'Einnahmen', 'Wendelinusheim', 'Konvention', 'Tannenwaldklinik', 'Zugestehen', 'Politikers', 'Schlager', 'Bürokraten', 'Beschäftigte', 'Zweiten', 'Quatsch', 'Boatpeople', 'Gleichung', 'Separieren', 'Notwendigkeit', 'Gedanke', 'Belegen', 'Pendeln', 'Stangen', 'Verantwortlichen', 'Konflikten', 'Autoren', 'Problems', 'Phantasien', 'Vorwarnung', 'Belegung', 'Puttrich'
                  ,'Spektrum', 'Schutzbedürftige', 'Hässliche', 'Aufenthalts', 'Ausführung', 'Befürworter', 'Abstieg', 'Unsicherheit', 'Umgehung', 'Dutzend', 'Vortrag', 'Vermittler', 'Schätzer', 'Hüterin', 'Antanzen', 'Ressort', 'Getöteter', 'Korrektur', 'Statuen', 'Organen', 'Schlosses', 'Klärung', 'Zurzeit', 'Konglomerat', 'Verstimmungen', 'Alterung', 'Horizont', 'Banlieue', 'Verweigerer', 'Hotspots', 'Antieuropäisch', 'Illegalität', 'Auswirkung', 'Verbrennung', 'Verschweigen', 'Durchwinkens', 'Erwerbstätige', 'Experiment', 'Hochqualifizierte', 'Kantonisten', 'Wanderschaft', 'Banlieues', 'Vertrauens', 'Einstellen', 'Helligkeit', 'Ministers', 'Konsulat', 'Kreises', 'Kriftel', 'Mathematik', 'Instituts', 'Klientel', 'Konzerns', 'Badminton', 'Ressourcen', 'Wahrnehmungen', 'Rückzieher', 'Korrektur', 'Heizung', 'Schlachten', 'Verpflichtungen', 'Vereinen', 'Lektion', 'Geläutert', 'Banalität', 'Ominösen', 'Übereinkunft', 'Register', 'Feuerlegen', 'Zähmung', 'Zumutungen', 'Erratischer', 'Streitereien', 'Zuversicht', 'Garagen', 'Gauland', 'Rücklage', 'Unterschied', 'Soziologen', 'Motivation', 'Kindheit', 'Reinigungen', 'Branche', 'Gefechte', 'Zusammenhangs', 'Ereignisse', 'Ausgrenzung', 'Erwerbslosigkeit', 'Schwarzfahrer', 'Überweisen', 'Zertifikat', 'Getränke', 'Anstellung', 'Hexerei', 'Start-up', 'Informatiker', 'Reibach', 'Realismus', 'Honorar', 'Unterschied', 'Antagonismen', 'Emissäre', 'Zweifach', 'Erwerbstätiger', 'Alleinlassen', 'Sklaven', 'Partizipation', 'Verdichtung', 'Abrücken', 'Fastenbrechen', 'Besitzer', 'Rücklagen', 'Nielsen-Hochhaus', 'Schuhen', 'Ländcheshalle', 'Widerstreit', 'Projektil', 'Unerhörtes', 'Vorzeichen', 'Verhältnissen', 'Zuversicht', 'Einstieg', 'Schlossgrabenfest', 'Revolte', 'Verpflichtung', 'Vereinbarungen', 'Sängerin'
                  , 'Ausverkauf', 'Gesetzlicher', 'Vieldeutigkeit', 'Knechte', 'Gläubige', 'Verschuldung', 'Abhängigkeiten', 'Sabrata', 'Betonung', 'Stipendien', 'Gefangenen', 'Schritten', 'Entrüstung', 'Angesicht', 'Enttäuschungen', 'Besonderes', 'Programmen', 'Vorgabe', 'Siedlung', 'Remilitarisierung', 'Salzhemmendorfer', 'Flüchtlings-', 'Archtikturbiennale', 'Weiltal', 'Wetterau', 'Separate', 'Angelegenheiten', 'Westedt', 'Gärtnern', 'Wartens', 'Bützche', 'Unvorhergesehene', 'Coaches', 'Unterwanderung', 'Gesetzgebung', 'Gutleutstraße', 'Interpretation', 'Reserve', 'Konservativen', 'Schweißer', 'Platsch', 'Ankündigungen', 'Kartoffeln', 'Joghurt', 'Nachwuchs', 'Wildwuchs', 'Start-ups', 'Basketball', 'Gefährlichkeit', 'Demirezen', 'Komplikation', 'Erwärmung', 'Fassade', 'Beteiligungen', 'Polnisch', 'Reparatur', 'Unbegleitete', 'Absagen', 'Bavendamm', 'Subsidiärer', 'Deutschlehren', 'Nouruz-Feier', 'Aussetzung', 'Parallelen', 'Schmerz', 'Ernst-Reuter-Schule', 'Lehrerinnen', 'Musizieren', 'Gymnasiums', 'Mittelständler', 'Porträts', 'Irritationen', 'Anne-Frank-Wettbewerb', 'Elemente', 'Erhebung', 'Kandidat', 'Zittrigen', 'Gruppierungen', 'Dissonanzen', 'Erbitterter', 'Residenz', 'Investment', 'Orientierung', 'Kanadier', 'Benefiz', 'Schiffchen', 'Grindeanu', 'Abgrund', 'Abgründe', 'Vorhangs', 'Konservativen'
                  , 'Hinsicht', 'Haftung', 'Hoëcker', 'Wiederanstieg', 'provocatore', 'Idioten', 'Meisten', 'Wandlung', 'Halligen', 'Knatsch', 'Rabatte', 'Städtchen', 'Absicht'  
                  , 'Labsaal', 'Missionen', 'Geflohene', 'Schäden', 'Broschüren', 'Wehrhafte', 'Dunkeln', 'Betroffenheit', 'Layenhof', 'Abarbeiter', 'Aufstockung', 'Sondierung', 'Geheimnis', 'Redners', 'Lüftchen', 'Aufgeschlossenheit', 'Vorbote', 'Erwerbstätigen', 'Exorzismus', 'Grande-Synthe', 'Anonymität', 'Dozenten', 'Angemessen', 'Umleitung', 'Zerstörungen', 'Gekränkte', 'Dreiseitel', 'Dunkelheit', 'Besatzung', 'Ehrlich-Schule', 'Erzählung', 'Cricket', 'Inszenierung', 'Unhöflichkeiten', 'Pazifiks', 'Folterung', 'Einheimischer', 'Temperatur', 'Sklaverei', 'Deutscher', 'Autonomie', 'Mischung', 'Prozente', 'Böseste', 'Verbannung', 'Korrespondent', 'Argumentation', 'Vorzügen', 'Verleumdung', 'Erholung', 'Eigentümer', 'Bestätigung', 'Erich-Fromm-Preis', 'Liegenschaften', 'Workshop', 'Verhängung', 'Überlaufen', 'schrumpfen', 'Unabhängigkeit', 'Bürgschaft', 'Verbraucher', 'Davongeschlichen', 'Selbstbewusst', 'Schülern', 'Karriere', 'Missachtung', 'Profiteur', 'Schleusers', 'Wiedersehen', 'Haitianer', 'Humaner', 'Deckels', 'Kompetenz', 'Sympathien', 'Interventionen', 'Pforten', 'Selbstanschwärzung', 'Verweis', 'Thüringer', 'Maltesern', 'Wartende', 'Normalisierung', 'Variationen'
                  ,'Bilateralisierung', 'Vermächtnis', 'Sternchen', 'Einsteiger', 'Signalen', 'Konzentration', 'Schnelligkeit', 'Italienern', 'Bedürftigen', 'Anmaßung', 'Kontinents', 'Lausiger', 'Berufen', 'Qualifikationen', 'Immobilie', 'Walter-Dirks-Preis', 'Hoffnungsvoller', 'Inkonsequenz', 'Resettlement', 'Organisator', 'Karawanen', 'Verschwende', 'Zugewanderte', 'Anlagen', 'Instanz', 'Patient', 'Michaelisdorf', 'Kriterium', 'Gefangen', 'Verspätung', 'Behauptung', 'Narrativ', 'Dichter', 'Unterzeichner', 'Georgier', 'Entfernung', 'Lampedusa', 'Konzepten', 'Tätigen', 'Zerstören', 'Verfangen', 'Interesses', 'Genitiv', 'Abgeschobenen', 'Schwere', 'Sensation', 'Technologien', 'Legende', 'kooperieren', 'Abstammung', 'Egozentrik', 'Barmherzigkeit', 'Branchen', 'Auslieferung', 'Humanitarismus', 'Dopplungen', 'Vorreiter', 'Zweisamkeit', 'Schnecke', 'Kündigungen', 'Vermittlers', 'Traumatisierte', 'Beratern', 'Absurditäten', 'Gefährden', 'Kompromisse', 'Pentagon', 'Irreführung', 'Fünfjährige', 'Ungeduld', 'Aufmerksamkeit', 'Unterscheidung', 'Verlegungen', 'Duldungen', 'Misslingen', 'Osteuropäer', 'Überschreitung', 'Zusammenhänge', 'Genötigt', 'Wiederaufbau', 'Interviews', 'Beschuldigte', 'Reklame', 'Vergabe', 'Geballter', 'Gendarmerie', 'Shutdowns', 'Lektionen', 'Vertuschung', 'Verarmung', 'Sitzungen', 'Ausfahrt', 'Kompromissen', 'Sekunden', 'Ventile', 'Kategorischer', 'privaten'
                  ]
                  

long_nouns = [item for item in long_nouns if item not in items_to_remove]

print(len(long_nouns))
print(long_nouns)

668
['CSU-Vorsitzender', 'Ankerzentren', 'Grenzmanagement', 'Flüchtlingsunterkunft', 'Missstände', 'Hilfsgütern', 'Bürgerdialog', 'Kontrollstelle', 'Asylpolitik', 'EU-Ratsvorsitz', 'Rechtsverletzung', 'Europapolitik', 'Wasserschaden', 'Finanzhilfen', 'Flüchtlings-Unterbringungspolitik', 'Wanderungsbereitschaft', 'Helfertätigkeit', 'Volksparteien', 'Asylpolitik', 'Familiennachzug', 'Mittwoch', 'Marktwirtschaft', 'Wirtschaftsminister', 'KfW-Stiftung', 'Selbsthilfe', 'Flüchtlingsstatistik', 'Gewaltkriminalität', 'Rosenkranz', 'Rechtspopulisten', 'Unterschrift', 'Rückführungsabkommen', 'Symbolpolitik', 'Landweg', 'UN-Migrationspakt', 'Mittelstand', 'Führungskräfte', 'Bundestag', 'Asylstreit', 'Asylsuchenden', 'Systemkrise', 'Herkunftsländer', 'Bayern-Plan', 'Südwesten', 'Visegrád-Staaten', 'Migrationspolitik', 'Integrationsbemühungen', 'Gewaltanstieg', 'Unionspolitiker', 'Spurwechsel', 'Asylrecht', 'Integrationspolitik', 'EU-Ländern', 'Familiennachzug', 'Bamf-Präsident', 'Landrat', 'Bamf-B

In [17]:
#initializing a data frame
data = {'Noun': [], 'Title' : [], 'Year' : [], 'Source' : [], 'Annotation 0/1' : []}
df = pd.DataFrame(data)

for noun in long_nouns:
    for title in titles:
        if noun in title.split():
            new_data = {'Noun': [noun], 'Title': [title]}
            df = pd.concat([df, pd.DataFrame(new_data)], ignore_index=True)
            
print(df)

                       Noun  \
0          CSU-Vorsitzender   
1              Ankerzentren   
2              Ankerzentren   
3              Ankerzentren   
4     Flüchtlingsunterkunft   
...                     ...   
1715            Asylpolitik   
1716            Asylpolitik   
1717            Asylpolitik   
1718            Asylpolitik   
1719                OB-Wahl   

                                                  Title  Year  Source  \
0     Seehofer mahnt Merkel CSU-Vorsitzender fordert...   NaN     NaN   
1     Seehofer mahnt Merkel CSU-Vorsitzender fordert...   NaN     NaN   
2                            Einigung über Ankerzentren   NaN     NaN   
3     Koalition streitet weiter über Ankerzentren Kr...   NaN     NaN   
4     Kritik an Flüchtlingsunterkunft Stadt stellt P...   NaN     NaN   
...                                                 ...   ...     ...   
1715  Merkel will Vergabe von EU-Mitteln an Aufnahme...   NaN     NaN   
1716  Es kann jederzeit wieder losgehen Der

In [200]:
########### NOT NECESSARY #############################

#removing simplex nouns that i missed above
df = df[df['Noun'] != 'Humanität']
print(df[380:400])

                     Noun                                    Title
4648        Containerdorf        Ein Containerdorf für Flüchtlinge
4714  Stellvertreterkrieg                      Stellvertreterkrieg
4715          EU-Beitritt  EVP strikt gegen EU-Beitritt der Türkei
4716            Gedenktag                Gedenktag für Flüchtlinge
4782           Asylgesetz                  Asylgesetz im Bundesrat
4783            Bundesrat       Bundesrat ringt um Asyl-Kompromiss
4784            Bundesrat                  Asylgesetz im Bundesrat
5049     Kreuzfahrtschiff   Flüchtlings-Drama auf Kreuzfahrtschiff
5050     Kreuzfahrtschiff           Drama auf dem Kreuzfahrtschiff


## Storing the compounds in files

In [18]:
#writing the info from the data frame into a new file
#path of new file
csv_file = 'compTitleFAZ2018.csv'

with open(csv_file, 'w', newline='', encoding='utf-8') as csv_file:
    #writer = csv.writer(csv_file)
    #for item in df:Noun,Title,Year,Source,Annotation 0/1
    #    writer.writerow([item])
    df.to_csv(csv_file, index=False)

### removing duplicate titles

In [25]:
#remove duplicate entries
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/kaggle/input/comptitle-all-duplicates/compTitleSZ_all.csv')

# Drop duplicate rows based on the 'item' column
df.drop_duplicates(subset=['Noun', 'Title'], keep='first', inplace=True)

# Save the cleaned DataFrame back to a new CSV file
df.to_csv('compTitleSZ_all.csv', index=False)

# LOADING THE ANNOTATED FILE

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/kaggle/input/all_annotated.csv')